In [1]:
import os


In [7]:
pwd

'c:\\Users\\user\\Desktop\\End-to end- ML-project-with-ML-flow\\End-to-end-Machine-Learning-Project-with-Mlflow'

In [10]:
os.environ['MLFLOW_TRACKING_USERNAME'] = 'WisdomAnalyst'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '781aa8b98086477f57088d72da8f137a4686f662'
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/WisdomAnalyst/End-to-end-Machine-Learning-Project-with-Mlflow.mlflow'

In [44]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column:str 
    mlflow_uri:str


In [45]:
from Mlproject.constants import *
from Mlproject.utils.common import read_yaml, create_directories
import logging
import json
from pathlib import Path
from box import ConfigBox

logging.basicConfig(level=logging.INFO, format='[%(asctime)s]: %(message)s:')

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")
SCHEMA_FILE_PATH = Path("schema.yaml")


In [46]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = ConfigBox(read_yaml(config_filepath))
        self.params = ConfigBox(read_yaml(params_filepath))
        self.schema = ConfigBox(read_yaml(schema_filepath))

        logging.info(f"Config contents: {self.config}")
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation  
        params = self.params.ElasticNet  
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri='https://dagshub.com/WisdomAnalyst/End-to-end-Machine-Learning-Project-with-Mlflow.mlflow'
        )

        return model_evaluation_config

In [47]:
import os  
import pandas as pd  
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score  
from urllib.parse import urlparse  
import mlflow  
import mlflow.sklearn  
import numpy as np  
import joblib 
from typing import Any 


In [ ]:

def save_json(path: Path, data: Any):
    """Save data to a JSON file"""
    path = Path(path)
    with open(path, "w") as f:
        json.dump(data, f, indent=4)


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):  
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))  
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)  
        model = joblib.load(self.config.model_path)  

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)  
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme  

        with mlflow.start_run(run_name="ElasticNet_Wine_Quality_Model"):

            # Add tags
            mlflow.set_tag("model_type", "ElasticNet")
            mlflow.set_tag("data_source", "Wine Quality Dataset")
            mlflow.set_tag("experiment_description", "Baseline model with default parameters")

            # Log parameters
            mlflow.log_params({
                "alpha": self.config.all_params["alpha"],
                "l1_ratio": self.config.all_params["l1_ratio"],
                "data_version": "v1.0",
                "model_version": "ElasticNet_v1"
            })

            
            predicted_qualities = model.predict(test_x)
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)  

            # Log metrics to MLflow
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Log model to MLflow
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(
                    model, 
                    "model", 
                    registered_model_name="ElasticModel"  
                )
            else:
                mlflow.sklearn.log_model(model, "model")

In [49]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()  
except Exception as e:
    raise e

[2025-02-25 16:47:56,182: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-02-25 16:47:56,242: INFO: common: YAML file: params.yaml loaded successfully]
[2025-02-25 16:47:56,283: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-02-25 16:47:56,291: INFO: 1153941306: Config contents: {'artifacts_root': 'artifacts', 'data_ingestion': {'local_data_file': 'artifacts/data_ingestion/data.zip', 'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/WisdomAnalyst/FIFA-21-DATA-CLEANING-/raw/refs/heads/main/winequality-data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_transformation': {'data_path': 'artifacts/data_ingestion/winequality-red.csv', 'root_dir': 'artifacts/data_transformation', 'test_data_path': 'artifacts/data_transformation/test.csv', 'train_data_path': 'artifacts/data_transformation/train.csv'}, 'data_validation': {'STATUS_FILE': 'artifacts/data_validation/status.txt', 'root_dir': 'artifacts/data_validation', 'unzip_d

Registered model 'ElasticModel' already exists. Creating a new version of this model...
2025/02/25 16:48:24 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticModel, version 2
Created version '2' of model 'ElasticModel'.
